In [363]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import geopandas as gpd
from shapely.geometry import Point
import pickle
from datetime import datetime

import_new_data=False

## clean counters up

In [ ]:
if import_new_data==True:
    import_df = pd.read_csv('data/Bicycle_Counts.csv', dtype={'counts': int})
    import_df['date'] = pd.to_datetime(import_df['date'])
    import_df = import_df.set_index('date')
    
    date_retrieved = datetime.now()
    with open('data/retrieval_date.pkl', 'wb') as f:
        pickle.dump(date_retrieved, f)


'''
status col:
"0 = raw
1 = excluded 
2 = deleted
4 = modified
8 = validated
16 = certified"'''

In [442]:
# copy after imported to save having to import again
data = import_df[['id','counts']].copy()
data.loc[data['counts'].str.contains(',')==True, 'counts'] = data.loc[data['counts'].str.contains(',')==True, 'counts'].str.replace(',','')
data['counts'] = pd.to_numeric(data['counts']).astype(int)
data

,id,counts
date,,
2018-12-24 03:30:00,100009424,0
2018-12-24 03:45:00,100009424,0
2018-12-24 04:00:00,100009424,0
2018-12-24 04:15:00,100009424,0
2018-12-24 04:30:00,100009424,0
...,...,...
2023-09-12 22:45:00,300029648,18
2023-09-12 23:00:00,300029648,18
2023-09-12 23:15:00,300029648,15


In [443]:
counters = pd.read_csv('data/Bicycle_Counters.csv')
counters = counters[['id', 'name', 'latitude', 'longitude']]

In [444]:
# merge counter names
data = data.reset_index().merge(counters[['id', 'name']], on='id').set_index('date')

In [445]:
# drop counters that are tests
test_counters = [300020692, 100048744, 100005020, 100055175]
# Drop pedestrian counters
ped_counters = [300029648, 100009426]
# Drop other counters
# bk bridge comprehensive is all we need for the bridge
other_counters = [300020241, 100010022, 300028963, 100051865, 100009429, 100039064, 100057318, 100047029]


counters_to_remove = test_counters + ped_counters + other_counters

data = data[~data['id'].isin(counters_to_remove)]
counters = counters[~counters['id'].isin(counters_to_remove)]


#rename n.8th kent because we replace it with kent ave comprehensive below
counters.loc[counters['name']=='Kent Ave btw North 8th St and North 9th St', ['name', 'id',]] = ('Kent Ave Comprehensive', 999999999)
counters = counters[counters['name']!='Kent Ave btw South 6th St. and Broadway']

In [446]:
# get the first and last date of each counter
first_dates = data.reset_index().groupby('name')['date'].min()
last_dates = data.reset_index().groupby('name')['date'].max()

counter_dates = pd.DataFrame({'first': first_dates, 'last':last_dates}).sort_values(by='last')
counter_dates['runtime'] = (counter_dates['last'] - counter_dates['first']).dt.days
counter_dates.sort_values(by='runtime')

,first,last,runtime
name,,,
Willis Ave Bikes,2022-09-02 11:15:00,2023-09-12 23:45:00,375
Fountain Ave,2022-08-23 10:30:00,2023-09-12 23:45:00,385
111th St at 50th Ave,2022-05-09 15:00:00,2023-09-11 23:45:00,490
Kent Ave btw South 6th St. and Broadway,2014-08-14 00:00:00,2016-11-21 23:45:00,830
Amsterdam Ave at 86th St.,2019-10-24 00:00:00,2023-09-10 23:45:00,1417
Columbus Ave at 86th St.,2019-10-24 00:00:00,2023-09-11 23:45:00,1418
2nd Avenue - 26th St S,2015-05-22 09:15:00,2019-06-11 23:45:00,1481
8th Ave at 50th St.,2018-06-14 00:00:00,2023-09-11 23:45:00,1915
Pulaski Bridge,2017-06-24 00:00:00,2023-09-12 23:45:00,2271


In [447]:
# the two kent ave counters about 10 blocks from one another, let's combine them
data.loc[data['name']=='Kent Ave btw South 6th St. and Broadway', ['name', 'id']] = ('Kent Ave Comprehensive', 999999999)

data.loc[data['name']=='Kent Ave btw North 8th St and North 9th St', ['name', 'id']] = ('Kent Ave Comprehensive', 999999999)

### clean anomalous data

In [448]:
# several counters have odd spikes
# 8th ave and 50th st. reads zero for the first year and then spikes up to 22k in a single day throughout July
# could not find events that occurred in that July to cause a spike
# deleting everything before 7/24/2019
eighth_criteria = (data['id']==100057316) & (data.index < '2019-07-24')
data = data[~eighth_criteria].copy()

amsterdam_criteria = (data['id']==100057319) & (data.index < '2020-12-15')
data = data[~amsterdam_criteria].copy()

first_ave_criteria = (data['id']==100010020) & ((data.index<'2016-08-10') | (data.index>'2019-06-11'))
data = data[~first_ave_criteria].copy()

second_ave_criteria = (data['id']==100009424) & (data.index > '2018-07-14')
data = data[~second_ave_criteria].copy()

data = data.sort_index()

### replace with pretty names

In [449]:
pretty_names = {
    '2nd Avenue - 26th St S': '26th St & 2nd Ave', 
    'Prospect Park West':'Prospect Pk W',
    'Williamsburg Bridge Bike Path': 'Williamsburg Br',
    'Ed Koch Queensboro Bridge Shared Path':'Queensboro Br', 
    'Staten Island Ferry':'Staten Isl Ferry St',
    'Pulaski Bridge':'Pulaski Br', 
    '1st Avenue - 26th St N - Interference testing': '1st Ave & 26th St',
    '8th Ave at 50th St.': '8th Ave & 50th St',
    'Amsterdam Ave at 86th St.':'Amsterdam Ave & 86th St', 
    'Manhattan Bridge Bike Comprehensive':'Manhattan Br',
    'Comprehensive Brooklyn Bridge Counter':'Brooklyn Br', 
    'Fountain Ave':'Fountain Ave',
    'Columbus Ave at 86th St.':'Columbus Ave & 86th St',
    'Kent Ave Comprehensive': 'Kent Ave', 
    '111th St at 50th Ave':'111th St & 50th Ave',
    'Willis Ave Bikes':'Willis Ave'
}

In [450]:
counters['name'] = counters['name'].map(pretty_names)
data['name'] = data['name'].map(pretty_names)

In [451]:
counters

,id,name,latitude,longitude
1,100010018,Pulaski Br,40.742563,-73.951492
3,999999999,Kent Ave,40.720959,-73.960930
8,300024007,111th St & 50th Ave,40.745630,-73.852500
10,100009424,26th St & 2nd Ave,40.739710,-73.979540
11,300020904,Brooklyn Br,40.711644,-74.004109
14,100057316,8th Ave & 50th St,40.762348,-73.986120
15,100009427,Williamsburg Br,40.710530,-73.961450
16,100057320,Columbus Ave & 86th St,40.787700,-73.975050
18,100057319,Amsterdam Ave & 86th St,40.787700,-73.975050
20,100010020,1st Ave & 26th St,40.738830,-73.977165


In [452]:
counters.set_index('id').to_pickle('data/counters.pkl')

# groupby hour

In [453]:
# first aggregate up the 15 min counts to the hour level
# group by week but preserve the year so we have all historical data
historical_hr = data.reset_index()[['date', 'id', 'counts', 'name']]\
                    .groupby([pd.Grouper(key='date', freq='H'), 'id', 'name'])\
                    .sum()\
                    .reset_index()

In [455]:
# get hour of day for display
def get_time(hour):
    hour_string = str(hour).zfill(2)
    time_string = hour_string + ':00' + ':00'
    return pd.to_datetime(time_string, format='%H:%M:%S', utc=True)

In [456]:
#groupby hour and take mean
hr = historical_hr.groupby([historical_hr['date'].dt.hour, 'id', 'name']).mean()
hr['display_time'] = hr.index.get_level_values('date').to_series().apply(lambda x: get_time(x)).values

In [457]:
hr.to_pickle('data/by_hour.pkl')

## groupby week

In [458]:
# group by week but preserve the year so we have all historical data
historical_wk = data.reset_index()[['date', 'id', 'counts', 'name']]\
                    .groupby([pd.Grouper(key='date', freq='W-MON'), 'id', 'name'])\
                    .sum()

In [459]:
# get month of year given week number
def get_month(week_number, year):
    first_day = datetime.fromisocalendar(year, week_number, 1)
    return first_day.strftime('%m-%d')

In [460]:
historical_wk_noidx = historical_wk.reset_index()
wk = historical_wk_noidx.groupby([historical_wk_noidx['date'].dt.week,'id', 'name']).mean()
wk = wk.loc[wk.index.get_level_values('date')!=53]
wk['display_date'] = wk.index.get_level_values('date').to_series().apply(lambda x: get_month(x,2021)).values

/var/folders/4p/cnm8c74x2cscn3bny0pr_bvr0000gn/T/ipykernel_82121/809708603.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  wk = historical_wk_noidx.groupby([historical_wk_noidx['date'].dt.week,'id', 'name']).mean()


In [461]:
wk.to_pickle('data/by_week.pkl')

## make app data

In [462]:
# to save time on the color mapping, map colors ahead of time

# colors
cat_20 = ['#1f77b4',
    '#aec7e8',
    '#ff7f0e',
    '#ffbb78',
    '#2ca02c',
    '#98df8a',
    '#d62728',
    '#ff9896',
    '#9467bd',
    '#c5b0d5',
    '#8c564b',
    '#c49c94',
    '#e377c2',
    '#f7b6d2',
    '#7f7f7f',
    '#c7c7c7',
    '#bcbd22',
    '#dbdb8d',
    '#17becf',
    '#9edae5'
    ]

num_counters = len(counters)
counter_ids = counters['id'].unique()
color_indices = np.linspace(0, len(cat_20)-1, num_counters, dtype=int)
colors = [cat_20[x] for x in color_indices]
color_dict = dict(zip(counter_ids, colors))

In [463]:
counter_display = counters.copy()
counter_display['color'] = counter_display['id'].map(color_dict)
counter_display.set_index('id').to_pickle('bike-counts-app/data/streamlit_counters.pkl')

hr_display = hr.copy()
hr_display['color'] = hr_display.index.get_level_values('id').map(color_dict)
hr_display.to_pickle('bike-counts-app/data/streamlit_by_hr.pkl')

wk_display = wk.copy()
wk_display['color'] = wk_display.index.get_level_values('id').map(color_dict)
wk_display.to_pickle('bike-counts-app/data/streamlit_by_wk.pkl')

historical_wk_display = historical_wk.copy()
historical_wk_display['id'] = historical_wk_display.index.get_level_values('id').astype(int)
historical_wk_display['color'] = historical_wk_display['id'].map(color_dict)
historical_wk_display = historical_wk_display.drop(columns=['id'])
cutoff = '2015-01-01'
historical_wk_display = historical_wk_display.loc[historical_wk_display.index.get_level_values('date') >= cutoff ]
historical_wk_display.to_pickle('bike-counts-app/data/streamlit_hist_by_wk.pkl')

data.to_pickle('bike-counts-app/data/all_data.pkl')

In [386]:
hr_display

date  \
date id        name                                                
0    100009424 26th St & 2nd Ave   2016-11-17 04:31:34.388224512   
     100009425 Prospect Pk W       2020-04-09 02:30:41.860465152   
     100009427 Williamsburg Br     2018-10-19 13:34:35.912408832   
     100009428 Queensboro Br       2018-10-21 09:47:19.550561792   
     100010017 Staten Isl Ferry St 2019-12-19 06:07:01.657458432   
...                                                          ...   
23   300020904 Brooklyn Br         2018-11-01 16:42:10.087744256   
     300024007 111th St & 50th Ave 2023-01-07 11:37:06.804123648   
     300027723 Fountain Ave        2023-03-03 20:03:45.000000256   
     300029647 Willis Ave          2023-03-08 12:53:53.243967744   
     999999999 Kent Ave            2019-03-03 04:27:10.415025920   

                                          display_time    color  
date id        name                                              
0    100009424 26th St & 2nd Ave   1900-01-01 00:00:00  #ffbb78  
     100009425 Prospect Pk W       1900-01-01 00:00:00  #f7b6d2  
     100009427 Williamsburg Br     1900-01-01 00:00:00  #ff9896  
     100009428 Queensboro Br       1900-01-01 00:00:00  #e377c2  
     100010017 Staten Isl Ferry St 1900-01-01 00:00:00  #bcbd22  
...                                                ...      ...  
23   300020904 Brooklyn Br         1900-01-01 23:00:00  #98df8a  
     300024007 111th St & 50th Ave 1900-01-01 23:00:00  #ff7f0e  
     300027723 Fountain Ave        1900-01-01 23:00:00  #dbdb8d  
     300029647 Willis Ave          1900-01-01 23:00:00  #9edae5  
     999999999 Kent Ave            1900-01-01 23:00:00  #aec7e8  

[384 rows x 3 columns]